In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pandas as pd
import numpy as np
from datasurfer.lib_objects.pdf_object import PDFPagesObject
from datasurfer.lib_objects.docx_object import DOCXObject
from datasurfer.datautils import xml_valid_df

In [17]:
pobj = PDFPagesObject(r'D:\02_Translation\01_Künstliche_Intelligenz_für_Ingenieure\KII.pdf', range(157-1, 174))
fdst = 'KII_Chapter_05_Original.xlsx'

In [18]:
annos = ['© Walter de Gruyter GmbH Berlin Boston. All rights reserved. \n',
 'This work may not be translated, copied or made publicly accessible in whole or part without the written permission of the publisher  \n',
 '(Walter De Gruyter GmbH, Genthiner Straße 13, 10785 Berlin, Germany). \n',
 'Vertraulich\n']

In [19]:
df = pobj.df[~pd.concat([pobj.df.text == s for s in annos], axis=1).any(axis=1).values]

df

,x0,y0,x1,y1,text,index,type,page_num
0,70.944000,37.439972,73.439857,48.479969,\n,0,0,156
1,70.944000,795.239929,73.439857,806.279968,\n,1,0,156
2,70.944000,50.879974,122.995857,102.260002,\n \n \n \n \n,2,0,156
6,48.168003,444.284119,59.208000,446.779968,\n,6,0,156
7,61.728004,822.336121,72.768005,824.831970,\n,7,0,156
...,...,...,...,...,...,...,...,...
434,121.878395,376.264069,490.264404,398.607330,•\nSimulationsuntersuchung eines dynamischen S...,15,0,173
435,121.887367,401.341980,490.434784,423.685242,•\nStabilitätsanalyse: Für gegebene Systemmatr...,16,0,173
436,121.905296,426.419891,490.461578,513.793579,"•\nModell des Regelkreises: Für das Modell (A,...",17,0,173
437,113.333603,554.776245,204.789368,568.500610,Literaturhinweise\n,18,0,173


In [20]:

df = df[df.text.str.strip().str.len()>0].reset_index(drop=True)

df['rel_pagenum'] = df.page_num - df.page_num.min()

df

,x0,y0,x1,y1,text,index,type,page_num,rel_pagenum
0,426.054108,213.080688,481.844788,312.706879,5\n,8,0,156,0
1,160.493607,315.166351,481.741333,367.154449,Wissensverarbeitung mit strukturierten\nObjekt...,9,0,156,0
2,121.340965,409.581757,466.879791,470.532959,Durch die Repräsentation von Wissen als strukt...,10,0,156,0
3,104.933502,515.656128,340.877411,529.380493,5.1 Begriffsbildung und strukturierte Objekte\n,11,0,156,0
4,104.933502,540.621704,482.352142,673.189392,Begriffe und Methoden existieren im menschlich...,12,0,156,0
...,...,...,...,...,...,...,...,...,...
270,121.878395,376.264069,490.264404,398.607330,•\nSimulationsuntersuchung eines dynamischen S...,15,0,173,17
271,121.887367,401.341980,490.434784,423.685242,•\nStabilitätsanalyse: Für gegebene Systemmatr...,16,0,173,17
272,121.905296,426.419891,490.461578,513.793579,"•\nModell des Regelkreises: Für das Modell (A,...",17,0,173,17
273,113.333603,554.776245,204.789368,568.500610,Literaturhinweise\n,18,0,173,17


In [21]:
df[df.rel_pagenum == 1]

,x0,y0,x1,y1,text,index,type,page_num,rel_pagenum
5,113.333603,117.544724,490.219757,129.326050,134\n5 Wissensverarbeitung mit strukturierten ...,8,0,157,1
6,113.333603,144.020844,490.821930,204.854904,strakte Darstellungsform „Objekt“ verwendet wi...,9,0,157,1
7,113.333611,229.015167,368.087433,240.452698,5.1.1 Begriffshierarchien und Vererbung von Ei...,10,0,157,1
8,113.333611,248.652252,490.832001,345.353119,Die Bildung von Begriffen ist ein wichtiges Hi...,11,0,157,1
9,113.343559,355.425690,490.712524,416.379333,Taxonomien. Die für einen Gegenstandsbereich g...,12,0,157,1
10,286.847900,437.799164,330.094330,453.694336,      \n   \n  \n,13,0,157,1
11,286.382507,491.964569,427.506104,503.062561,         \n\r  \n,14,0,157,1
12,382.004089,531.610413,421.859558,540.470459,    \n  \n,15,0,157,1
13,305.248901,550.533447,457.985840,570.365540,\n   \n\t          \n\n  ...,16,0,157,1
14,259.961914,529.339844,320.857697,538.289856,\n           \n,17,0,157,1


In [22]:

def remove_title_line(df):
    count = 0
    for i, (text, num) in df[['text', 'rel_pagenum']].iterrows():  
        if num > count:
            count += 1       
        else:        
            yield([text, num])
        
out0 = list(remove_title_line(df))



In [23]:
def split_paragraph(text, threshold=0.8):
    
    lst_text = np.array([txt for txt in text.split('\n') if txt.strip()])
    larr = np.array([len(txt) for txt in lst_text])
    isnewline = larr < (larr.max() * threshold)
    
    out = ''
    for isnl, txt in zip(isnewline, lst_text):
        if isnl:
            out += txt
            yield out
            out = ''
        else:
            out += txt.rstrip('-')
            
    if len(out) > 0:      
        yield out

    


In [24]:

def merge_paragraphs(out0):
    count = 0 

    for text, num in out0:
        for txt in split_paragraph(text):
            yield [txt, num]
            
out1 = list(merge_paragraphs(out0))

In [25]:

def merge_paragraphs(out1):
    
    iterator = iter(out1)
    line0, num0 = next(iterator)
    while True:
        try:       
            line1, num1 = next(iterator)
        except StopIteration:
            yield line0, num0
            break
        if num1 > num0:
        
            if not line1[0].isupper() and line0.strip()[-1] not in '.!?:;':
                line = line0 + ' ' + line1
                yield line, num0
                try:
                    line0, num0 = next(iterator)
                except StopIteration:
                    yield line0, num0
                    break
            else:
                yield line0, num0
                line0 = line1
                num0 = num1            
        else:
            yield line0, num0   
            line0 = line1
            num0 = num1
            
out2 = list(merge_paragraphs(out1))

In [26]:
dfout = pd.DataFrame(out2, columns=['Original', 'Page'])
dfout['Translation'] =  None
dfout['Review'] = 4 

dfout = xml_valid_df(dfout[['Page', 'Original', 'Translation', 'Review']])
dfout.Page = dfout.Page.astype('int') + df.page_num.min() + 1

dfout = dfout[~dfout['Original'].str.isnumeric()]
dfout = dfout[~(dfout['Original'].str.strip().str.len()<2)]

In [27]:
dfout.head()

,Page,Original,Translation,Review
1,157,Wissensverarbeitung mit strukturiertenObjekten,None,4
2,157,Durch die Repräsentation von Wissen als strukt...,None,4
3,157,5.1 Begriffsbildung und strukturierte Objekte,None,4
4,157,Begriffe und Methoden existieren im menschlich...,None,4
5,157,In diesem Kapitel werden mit semantischen Netz...,None,4


In [28]:
dfout.to_excel(fdst, index=False)